In [3]:
import pyarrow as pa

from pygdf.dataframe import DataFrame

import pandas as pd
import json
import numpy as np
import pygdf
from pygdf import DataFrame
import pickle

## Helper functions

In [4]:
def readArrowToDF(source):
    reader = pa.RecordBatchStreamReader(source)
    pa_df = reader.read_all()
    return pa_df.to_pandas()

def saveToDiskArrow(df, destination):
    if type(df) == pygdf.dataframe.DataFrame:
        pa_df = pa.RecordBatch.from_pandas(df.to_pandas())
    else:
        pa_df = pa.RecordBatch.from_pandas(df)
    path = destination+".arrow"
    file = open(path, 'wb')
    writer = pa.ipc.RecordBatchStreamWriter(file, pa_df.schema)
    writer.write_batch(pa_df)
    writer.close()
    file.close()

def get_ipc_handle_buffer(gdf_df):
    #pickle_dumps is a list of all column buffers
    pickle_dumps = []

    #pygdf df named -> gdf_df
    for i in gdf_df.columns:
        the_ipc_array_handle = gdf_df[i].to_gpu_array().get_ipc_handle()
        pickle_dumps.append(pickle.dumps(the_ipc_array_handle))
    
    return str(pickle_dumps)


def saveToDiskIPC(df,buffer,dataset_name):
    buff_location = '/usr/src/app/data/'+dataset_name+'.pickle'
    col_buff_location = '/usr/src/app/data/'+dataset_name+'-col.pickle'
    with open(buff_location, 'wb') as handle:
        pickle.dump(buffer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(col_buff_location,'wb') as handle:
        pickle.dump(df.columns, handle, protocol=pickle.HIGHEST_PROTOCOL) 

## pygdf dataframe

Required columns for mortgage dataframe: 

<b>zip, dti, current_actual_upb, borrower_credit_score, load_id, seller_name(integer mapping), delinquency_12_prediction

In [ ]:
required_columns = ['zip', 'dti', 'current_actual_upb', 
                    'borrower_credit_score', 'load_id', 'seller_name', 'delinquency_12_prediction']

In [ ]:
gdf = gdf.loc[:,required_columns]

## Option 1: saving it to disk as an arrow file

In [ ]:
saveToDiskArrow(gdf, '/usr/src/app/data/predictions')

## Option 2: saving cuda IPC buffer as pickle

In [ ]:
saveToDiskIPC(gdf, get_ipc_handle_buffer(gdf), '/usr/src/app/data/predictions')